In [1]:
# install the necessary libraries 
# !pip install crewai crewai_tools langchain_community
# Warning control
import warnings
warnings.filterwarnings('ignore')


- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew, LLM

In [3]:
choose_LLM = 1
colab_exp = 0
import os

from langchain_openai import ChatOpenAI

if choose_LLM==1:
    if(colab_exp):
        from google.colab import userdata    ###for colab use
        llm = ChatOpenAI(model = "gpt-4o-mini",openai_api_key=userdata.get('OPENAI_API_KEY'))
    else:
        from dotenv import load_dotenv
        load_dotenv() 
        llm = "gpt-4o-mini" #ChatOpenAI(model = "gpt-4o-mini",openai_api_key=os.getenv("OPENAI_API_KEY"))

else:
    llm = ChatOpenAI(model="ollama/llama3.2", base_url="http://localhost:11434",temperature=0.1)

    
# else:
#     from langchain_community.llms import HuggingFaceHub
#     llm = HuggingFaceHub(
#         repo_id="HuggingFaceH4/zephyr-7b-beta",
#         huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
#         task="text-generation",
#     )

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm= llm,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm= llm,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm= llm,
    verbose=True
)

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=1
)

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see here.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
### Content Plan Document: Exploring Artificial Intelligence

**1. Latest Trends in Artificial Intelligence**
   - **Generative AI**: Advancement in models like OpenAI's ChatGPT and Google's Bard, showcasing capabilities in content creation and human-like interactions.
   - **AI in Automation**: Increased integration of AI in workplace automation, particularly in tasks such as customer service (chatbots), data entry, and predictive analytics.
   - **Ethical AI**: Growing focus on responsible AI, transparency, and minimizing biases in AI algorithms, with companies like IBM leading initiativ

In [12]:
from IPython.display import display, Markdown
Markdown(result.raw)

```markdown
# The Rise of Artificial Intelligence: Transforming Our Future

## I. Introduction

Artificial Intelligence (AI) has gradually transitioned from the realm of science fiction to a central fixture in our everyday lives. With rapid advancements in computational power and algorithmic design, AI technologies are redefining our methods of communication, work practices, and decision-making processes. Understanding the current trends and implications of AI is crucial, not only for technology enthusiasts and professionals but for everyone living in a world increasingly shaped by automated systems.

The swift rise of AI not only promises to enhance productivity but also raises significant questions regarding ethics, bias, and job security. This piece delves into the most significant trends in AI, highlights the key players driving innovation, and discusses recent advancements that are shaping the future of this cutting-edge field. Ultimately, it encourages an informed perspective on the integration of AI into our lives and underscores the importance of responsible adoption.

## II. Current Trends in AI

One of the most significant trends currently shaping the landscape of Artificial Intelligence is the emergence of generative AI. Platforms such as OpenAI's ChatGPT and Google's Bard exemplify how AI is evolving to create human-like content dynamically. Whether it involves writing articles, composing music, or synthesizing artwork, generative AI showcases capabilities that can effectively complement human creativity. The efficiency and effectiveness of these technologies hold the potential to transform industries, introducing new methodologies in content creation.

Another noteworthy trend is the adoption of AI in workplace automation. AI technologies such as chatbots and predictive analytics are increasingly deployed to streamline workflow processes, enhance customer service, and reduce repetitive tasks. This growing reliance on AI across sectors like finance, retail, and healthcare offers businesses opportunities to optimize their operations while also raising concerns about workforce displacement. Addressing these implications is essential for achieving a balance between technological advancement and the needs of the human workforce, ensuring that the benefits of AI are widely shared.

The ethical considerations surrounding AI merit significant attention as well. Biases inherent in AI algorithms can lead to skewed and unfair outcomes. Companies like IBM have stepped up efforts to promote transparency and responsibility in AI development. The emergence of ethical AI practices is crucial for instilling public trust and fostering accountability among developers and organizations that deploy AI systems.

## III. Key Players

The competitive landscape of Artificial Intelligence is characterized by several influential companies that drive advancements. OpenAI stands out as a pioneer in generative AI, revolutionizing content production with innovations like the latest ChatGPT, which boasts enhanced multi-modal capabilities. These developments have made significant impacts across various sectors, prompting new applications and uses for AI technologies.

Google DeepMind has made remarkable strides in AI research, particularly in reinforcement learning, highlighting the potential of AI to address complex problems. Meanwhile, IBM’s Watson emphasizes the importance of ethical AI development, using its expertise to guide organizations toward responsible AI practices. Microsoft integrates these capabilities into its array of products, seamlessly embedding AI into tools designed to enhance productivity. Additionally, NVIDIA plays a critical role by providing the essential hardware infrastructure supporting AI processing, thus further advancing the field.

## IV. Noteworthy News

Recent months have witnessed groundbreaking developments in AI technologies. In September 2023, OpenAI introduced a new version of ChatGPT, improving its ability to engage users through diverse content forms. This upgrade not only enhances user experience but also widens the scope of applications available for businesses and individual users alike.

In October 2023, Google showcased its AI innovations in health diagnostics at a prominent tech conference, revealing how AI can optimize and enhance medical assessments. Further, in November 2023, IBM announced partnerships aimed at establishing responsible AI regulations, reflecting a collective recognition of the importance of ethical stewardship in this rapidly evolving field.

## V. Target Audience Insights

This blog aims to engage various audience segments, including technology professionals, business leaders, students, educators, and the general public. Each group possesses unique interests concerning AI. For technology professionals, understanding the latest advancements and integration challenges is vital for staying current with industry innovations. Business leaders are particularly interested in insights regarding AI’s role in optimizing operations, while students and educators are increasingly captivated by the capabilities of AI and its implications for education.

Concerns regarding job displacement due to AI and misinformation surrounding its limitations resonate widely, underscoring the need for clear and factual discussions. By addressing these concerns, this content aims to provide clarity and insights into the inclusive management of AI within our societies, fostering a more informed public dialogue.

## VI. Conclusion

As the impact of Artificial Intelligence on society continues to evolve, it is essential to recognize both the opportunities and challenges presented by this transformative technology. A balanced approach—one that prioritizes ethical considerations and addresses its implications on employment and potential misinformation—is necessary for future advancements in AI.

It is evident that AI is poised to transform countless aspects of our lives, from healthcare processes to everyday decision-making. Engaging with these technologies responsibly will empower individuals and organizations to harness the benefits of AI while being mindful of its broader implications on society.

## VII. Call to Action

What are your thoughts on the rise of Artificial Intelligence? We encourage readers to share their insights, questions, or concerns regarding AI in the comments below. For those eager to deepen their understanding, be sure to explore the resources and forums dedicated to the ongoing exploration of AI technologies and their evolving role in our world.
```

- Display the results of your execution as markdown in the notebook.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Digital Twin"
# result = crew.kickoff(inputs={"topic": topic})

In [14]:
from IPython.display import display, Markdown
display(Markdown(result.raw))
# result.raw

```markdown
# The Rise of Artificial Intelligence: Transforming Our Future

## I. Introduction

Artificial Intelligence (AI) has gradually transitioned from the realm of science fiction to a central fixture in our everyday lives. With rapid advancements in computational power and algorithmic design, AI technologies are redefining our methods of communication, work practices, and decision-making processes. Understanding the current trends and implications of AI is crucial, not only for technology enthusiasts and professionals but for everyone living in a world increasingly shaped by automated systems.

The swift rise of AI not only promises to enhance productivity but also raises significant questions regarding ethics, bias, and job security. This piece delves into the most significant trends in AI, highlights the key players driving innovation, and discusses recent advancements that are shaping the future of this cutting-edge field. Ultimately, it encourages an informed perspective on the integration of AI into our lives and underscores the importance of responsible adoption.

## II. Current Trends in AI

One of the most significant trends currently shaping the landscape of Artificial Intelligence is the emergence of generative AI. Platforms such as OpenAI's ChatGPT and Google's Bard exemplify how AI is evolving to create human-like content dynamically. Whether it involves writing articles, composing music, or synthesizing artwork, generative AI showcases capabilities that can effectively complement human creativity. The efficiency and effectiveness of these technologies hold the potential to transform industries, introducing new methodologies in content creation.

Another noteworthy trend is the adoption of AI in workplace automation. AI technologies such as chatbots and predictive analytics are increasingly deployed to streamline workflow processes, enhance customer service, and reduce repetitive tasks. This growing reliance on AI across sectors like finance, retail, and healthcare offers businesses opportunities to optimize their operations while also raising concerns about workforce displacement. Addressing these implications is essential for achieving a balance between technological advancement and the needs of the human workforce, ensuring that the benefits of AI are widely shared.

The ethical considerations surrounding AI merit significant attention as well. Biases inherent in AI algorithms can lead to skewed and unfair outcomes. Companies like IBM have stepped up efforts to promote transparency and responsibility in AI development. The emergence of ethical AI practices is crucial for instilling public trust and fostering accountability among developers and organizations that deploy AI systems.

## III. Key Players

The competitive landscape of Artificial Intelligence is characterized by several influential companies that drive advancements. OpenAI stands out as a pioneer in generative AI, revolutionizing content production with innovations like the latest ChatGPT, which boasts enhanced multi-modal capabilities. These developments have made significant impacts across various sectors, prompting new applications and uses for AI technologies.

Google DeepMind has made remarkable strides in AI research, particularly in reinforcement learning, highlighting the potential of AI to address complex problems. Meanwhile, IBM’s Watson emphasizes the importance of ethical AI development, using its expertise to guide organizations toward responsible AI practices. Microsoft integrates these capabilities into its array of products, seamlessly embedding AI into tools designed to enhance productivity. Additionally, NVIDIA plays a critical role by providing the essential hardware infrastructure supporting AI processing, thus further advancing the field.

## IV. Noteworthy News

Recent months have witnessed groundbreaking developments in AI technologies. In September 2023, OpenAI introduced a new version of ChatGPT, improving its ability to engage users through diverse content forms. This upgrade not only enhances user experience but also widens the scope of applications available for businesses and individual users alike.

In October 2023, Google showcased its AI innovations in health diagnostics at a prominent tech conference, revealing how AI can optimize and enhance medical assessments. Further, in November 2023, IBM announced partnerships aimed at establishing responsible AI regulations, reflecting a collective recognition of the importance of ethical stewardship in this rapidly evolving field.

## V. Target Audience Insights

This blog aims to engage various audience segments, including technology professionals, business leaders, students, educators, and the general public. Each group possesses unique interests concerning AI. For technology professionals, understanding the latest advancements and integration challenges is vital for staying current with industry innovations. Business leaders are particularly interested in insights regarding AI’s role in optimizing operations, while students and educators are increasingly captivated by the capabilities of AI and its implications for education.

Concerns regarding job displacement due to AI and misinformation surrounding its limitations resonate widely, underscoring the need for clear and factual discussions. By addressing these concerns, this content aims to provide clarity and insights into the inclusive management of AI within our societies, fostering a more informed public dialogue.

## VI. Conclusion

As the impact of Artificial Intelligence on society continues to evolve, it is essential to recognize both the opportunities and challenges presented by this transformative technology. A balanced approach—one that prioritizes ethical considerations and addresses its implications on employment and potential misinformation—is necessary for future advancements in AI.

It is evident that AI is poised to transform countless aspects of our lives, from healthcare processes to everyday decision-making. Engaging with these technologies responsibly will empower individuals and organizations to harness the benefits of AI while being mindful of its broader implications on society.

## VII. Call to Action

What are your thoughts on the rise of Artificial Intelligence? We encourage readers to share their insights, questions, or concerns regarding AI in the comments below. For those eager to deepen their understanding, be sure to explore the resources and forums dedicated to the ongoing exploration of AI technologies and their evolving role in our world.
```